In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
import numpy as np
import os
import time

import utils

import networks

import matplotlib.pyplot as plt

import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 40
r = 0.05
sigma = 0.25
T = 1
S_range = [0, 130]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)

# Build Neural Network

In [4]:
fnn = networks.FeedforwardNeuralNetwork(2, 50, 1, 3)
fnn.cuda()

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-2): 2 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [5]:
n_epochs = 100000
lossFunction = nn.MSELoss()
optimizer = optim.Adam(fnn.parameters(), lr=0.00002)

samples = {"pde": 50000, "bc":25000, "fc":25000}

# Modelling

- For each iteration in the training loop, we are sampling data for the three physical conditions of the PDE.
- Then we are calculating the loss three times on the same model, accumulating them into a combined objective function to be minimised for the Neural Network.
- The first loss is the differential equation loss. Here we are trying to minimise the PDE by calculating gradients and forming the PDE itself.
- The remaining losses are calculated for boundary value and initial value conditions for the PDE.
- Mean Squared Error loss function `nn.MSELoss()` is chosen as the criterion to be minimised and 
- Adam optimizer `nn.optim.Adam(lr=3e-5)` with a learning rate of 0.00003 is chosen for performing the weight updates.

In [6]:
loss_hist = []
start_time = time.time()


for epoch in range(n_epochs):
    
    i_st_train, i_v_train, bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData3(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    i_st_train = torch.from_numpy(i_st_train).float().requires_grad_().to(device)
    i_v_train = torch.from_numpy(i_v_train).float().to(device)
    
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)
    
    
    # PDE Round
    y1_hat = fnn(n_st_train)
    
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    # print(grads)
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    # print(grads2nd)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    
    # BC Round
    y21_hat = fnn(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y21_hat)
    
    # final condition loss
    fc_hat = fnn(i_st_train)
    fc_loss = lossFunction(i_v_train, fc_hat)
    
    
    # Backpropagation and Update
    optimizer.zero_grad()
    combined_loss = 0.8 * pde_loss + 0.001 * bc_loss + 0.001 * fc_loss
    combined_loss.backward()
    optimizer.step()
    mse_loss = pde_loss + bc_loss + fc_loss
    
    loss_hist.append(mse_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.6f}, fc loss: {fc_loss.item(): 5f}, L2 loss: {mse_loss.item():6f}, minimum loss: {min(loss_hist):.5f}')

end_time = time.time()
print('run time:', end_time - start_time)

0/100000 PDE Loss: 0.00027, BC Loss: 3969.172363, fc loss:  1783.238403, L2 loss: 5752.411133, minimum loss: 5752.41113
500/100000 PDE Loss: 0.00316, BC Loss: 2035.379028, fc loss:  813.045715, L2 loss: 2848.427979, minimum loss: 2848.42798
1000/100000 PDE Loss: 0.00428, BC Loss: 438.634644, fc loss:  182.124573, L2 loss: 620.763489, minimum loss: 620.76349
1500/100000 PDE Loss: 0.00091, BC Loss: 52.295654, fc loss:  170.160873, L2 loss: 222.457443, minimum loss: 222.45744
2000/100000 PDE Loss: 0.00022, BC Loss: 29.550224, fc loss:  185.996078, L2 loss: 215.546524, minimum loss: 215.54652
2500/100000 PDE Loss: 0.00024, BC Loss: 28.692659, fc loss:  185.191650, L2 loss: 213.884552, minimum loss: 213.88455
3000/100000 PDE Loss: 0.00022, BC Loss: 28.412632, fc loss:  183.427200, L2 loss: 211.840057, minimum loss: 211.84006
3500/100000 PDE Loss: 0.00020, BC Loss: 28.074532, fc loss:  181.280380, L2 loss: 209.355103, minimum loss: 209.35510
4000/100000 PDE Loss: 0.00029, BC Loss: 27.734022,

KeyboardInterrupt: 